## Data loading

In [7]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import json

import regex as re
from python_proj.utils.util import safe_save_fig, subtract_dict, Counter
import python_proj.utils.exp_utils as exp_utils

file_name = "dataset_90_days_started_11_07_23_preprocessed"
base_path = exp_utils.BASE_PATH
data_path = f'{base_path}/final_data/{file_name}.csv'
figure_base_path = f"{base_path}/figures/modelling/{file_name}/"

# Meta stuff.
pr_merged_key = 'PullRequestIsMerged'
ftc_key = 'SubmitterIsFirstTimeContributor'
seed_counter = Counter()


# Loads regular dataframe.
df: pd.DataFrame = pd.read_csv(filepath_or_buffer=data_path, header=0)
if 'Unnamed: 1' in df.columns:
    df = df.drop(['Unnamed: 1', "Project Name.1"], axis=1)
print(df.columns)

# Loads first-time contributor data frame.
ftc_df = df[df[ftc_key]]
print(f'{len(df)=}')
print(f'{len(ftc_df)=}')

# Creates sub-datasets.
metadata_fields = ['ID', 'Project Name',
                   'Submitter ID', 'PR Number', 'Closed At']
dependent_fields = [pr_merged_key]
independent_fields = [field for field in df.columns
                      if (field not in metadata_fields
                          and field not in dependent_fields
                          and field != ftc_key)]

df[independent_fields].describe()


Index(['Project Name', 'ID', 'Submitter ID', 'PR Number', 'Closed At',
       'PullRequestIsMerged', 'ControlIntegratedBySameUser',
       'ControlPullRequestHasComments', 'ControlHasHashTagInDescription',
       'IntraProjectSubmitterPullRequestSuccessRate',
       'EcosystemExperienceSubmitterPullRequestSuccessRate',
       'DependencyEcosystemExperienceSubmitterPullRequestSuccessRate',
       'InversedDependencyEcosystemExperienceSubmitterPullRequestSuccessRate',
       'SubmitterIsFirstTimeContributor',
       'ControlPullRequestHasCommentByExternalUser',
       'ln(1 + ControlPullRequestLifeTimeInMinutes)',
       'ln(1 + ControlNumberOfCommitsInPullRequest)',
       'ln(1 + ControlIntraProjectPullRequestExperienceOfIntegrator)',
       'ln(1 + IntraProjectSubmitterPullRequestSubmissionCount)',
       'ln(1 + IntraProjectSubmitterPullRequestCommentCount)',
       'ln(1 + SharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator)',
       'ln(1 + SharedExperiencePullReq

,IntraProjectSubmitterPullRequestSuccessRate,EcosystemExperienceSubmitterPullRequestSuccessRate,DependencyEcosystemExperienceSubmitterPullRequestSuccessRate,InversedDependencyEcosystemExperienceSubmitterPullRequestSuccessRate,ln(1 + ControlPullRequestLifeTimeInMinutes),ln(1 + ControlNumberOfCommitsInPullRequest),ln(1 + ControlIntraProjectPullRequestExperienceOfIntegrator),ln(1 + IntraProjectSubmitterPullRequestSubmissionCount),ln(1 + IntraProjectSubmitterPullRequestCommentCount),ln(1 + SharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator),...,ln(1 + SharedExperienceIssueSubmittedByIntegratorCommentedOnBySubmitter),ln(1 + SharedExperienceIssueDiscussionParticipationByIntegratorAndSubmitter),ln(1 + EcosystemExperienceSubmitterIssueSubmissionCount),ln(1 + EcosystemExperienceSubmitterIssueCommentCount),ln(1 + DependencyEcosystemExperienceSubmitterIssueSubmissionCount),ln(1 + DependencyEcosystemExperienceSubmitterIssueCommentCount),ln(1 + InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount),ln(1 + InversedDependencyEcosystemExperienceSubmitterIssueCommentCount),ln(1 + WeightedFirstOrderInDegreeCentrality),ln(1 + WeightedFirstOrderOutDegreeCentrality)
count,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,...,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06
mean,5.032730e-01,4.034645e-01,9.889740e-02,5.955645e-02,4.193296e-01,1.184218e-01,2.985197e-01,1.691039e-01,1.420826e-01,5.359085e-02,...,1.639801e-02,3.941424e-02,8.724590e-02,1.451003e-01,1.151620e-02,1.976215e-02,9.566235e-03,1.757943e-02,8.884955e-02,7.438409e-02
std,4.710991e-01,4.525011e-01,2.911891e-01,2.291140e-01,2.257136e-01,7.155493e-02,1.868852e-01,1.844545e-01,1.853726e-01,1.297009e-01,...,7.423245e-02,1.070026e-01,1.245128e-01,1.815019e-01,5.415120e-02,8.022241e-02,5.261483e-02,8.245254e-02,1.050997e-01,9.868875e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.363720e-01,7.525668e-02,1.658047e-01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.875000e-01,0.000000e+00,0.000000e+00,0.000000e+00,4.430126e-01,7.525668e-02,2.972018e-01,9.318519e-02,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,7.753347e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.426778e-02,3.550956e-02
75%,1.000000e+00,9.399033e-01,0.000000e+00,0.000000e+00,5.858949e-01,1.505134e-01,4.381723e-01,3.095545e-01,2.734346e-01,0.000000e+00,...,0.000000e+00,0.000000e+00,1.416667e-01,2.575606e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.366330e-01,1.119194e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

print(f'{len(train)=}, {len(test)=}')

train_predictors = train[independent_fields]
train_dependent = train[pr_merged_key]

test_predictors = test[independent_fields]
test_dependent = test[pr_merged_key]


len(train)=979694, len(test)=244924


## Model Creation

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, roc_auc_score


def calculate_metrics(predicted_labels, true_labels, sample_weights=None):
    return {
        "accuracy": accuracy_score(true_labels, predicted_labels, sample_weight=sample_weights),
        "precision": precision_score(true_labels, predicted_labels, sample_weight=sample_weights),
        "recall": recall_score(true_labels, predicted_labels, sample_weight=sample_weights),
        "f1": f1_score(true_labels, predicted_labels, sample_weight=sample_weights)
    }


def create_model(
    train_predictors: pd.DataFrame,
    train_labels: pd.Series,
    test_predictors: pd.DataFrame,
    test_labels: pd.Series
):
    # Model creation and predictions.
    random_state = seed_counter.get_next()
    print(f'{random_state=}')
    rf = RandomForestRegressor(
        n_estimators=100,
        random_state=random_state)

    rf.fit(train_predictors, train_labels)

    predictions = rf.predict(test_predictors)
    predictions = [pred >= 0.5 for pred in predictions]

    # F1 scores.
    conf = confusion_matrix(test_labels, predictions)

    metrics = calculate_metrics(predictions, test_labels)

    print(f'{metrics=}\n')

    # Other metrics.
    print(f'Confusion matrix:\n{conf}\n')
    print("Classification report:")
    print(classification_report(test_labels, predictions))

    return rf, metrics


In [14]:
import regex as re
from python_proj.utils.util import subtract_dict

# Creates full model.
full_rf_model, full_rf_metrics = create_model(
    train_predictors,
    train_dependent,
    test_predictors,
    test_dependent
)

print("Performance metrics full model:")
print(json.dumps(full_rf_metrics, indent=4))


random_state=43
metrics={'accuracy': 0.871899038068952, 'precision': 0.8964364000137307, 'recall': 0.9468766186677717, 'f1': 0.9209663889889493}

Confusion matrix:
[[ 30745  21119]
 [ 10256 182804]]

Classification report:
              precision    recall  f1-score   support

       False       0.75      0.59      0.66     51864
        True       0.90      0.95      0.92    193060

    accuracy                           0.87    244924
   macro avg       0.82      0.77      0.79    244924
weighted avg       0.87      0.87      0.87    244924

Performance metrics full model:
{
    "accuracy": 0.871899038068952,
    "precision": 0.8964364000137307,
    "recall": 0.9468766186677717,
    "f1": 0.9209663889889493
}


In [4]:

full_rf_metrics = {
    "accuracy": 0.871899038068952,
    "precision": 0.8964364000137307,
    "recall": 0.9468766186677717,
    "f1": 0.9209663889889493
}

In [15]:

# Creates control model.
control_fields = [field for field in independent_fields
                  if re.match(r'.*Control.*', field)]
print(control_fields)
control_rf_model, control_rf_metrics = create_model(
    train[control_fields],
    train_dependent,
    test[control_fields],
    test_dependent
)

print(json.dumps(control_rf_metrics, indent=4))
print("Comparison between control model and full model (negatives indicate control is better):")
diff = subtract_dict(full_rf_metrics, control_rf_metrics)
print(json.dumps(diff, indent=4))


['ControlIntegratedBySameUser', 'ControlPullRequestHasComments', 'ControlHasHashTagInDescription', 'ControlPullRequestHasCommentByExternalUser', 'ln(1 + ControlPullRequestLifeTimeInMinutes)', 'ln(1 + ControlNumberOfCommitsInPullRequest)', 'ln(1 + ControlIntraProjectPullRequestExperienceOfIntegrator)']
random_state=44
metrics={'accuracy': 0.8016731720860348, 'precision': 0.8522185980975784, 'recall': 0.9053972858178805, 'f1': 0.8780034508133522}

Confusion matrix:
[[ 21553  30311]
 [ 18264 174796]]

Classification report:
              precision    recall  f1-score   support

       False       0.54      0.42      0.47     51864
        True       0.85      0.91      0.88    193060

    accuracy                           0.80    244924
   macro avg       0.70      0.66      0.67    244924
weighted avg       0.79      0.80      0.79    244924

{
    "accuracy": 0.8016731720860348,
    "precision": 0.8522185980975784,
    "recall": 0.9053972858178805,
    "f1": 0.8780034508133522
}
Compar

In [11]:
# Creates measured information model.
non_control_fields = [field for field in independent_fields
                      if field not in control_fields]
non_control_rf_model, non_control_rf_metrics = create_model(
    train[non_control_fields],
    train_dependent,
    test[non_control_fields],
    test_dependent
)

print(json.dumps(non_control_rf_metrics, indent=4))
print("Comparison between non-control model and full model (negatives indicate non-control is better):")
diff = subtract_dict(full_rf_metrics, non_control_rf_metrics)
print(json.dumps(diff, indent=4))


random_state=43
metrics={'accuracy': 0.7925478924074407, 'precision': 0.8091427876027346, 'recall': 0.9642766500748445, 'f1': 0.8799243766985703}

Confusion matrix:
[[  7944  43913]
 [  6897 186170]]

Classification report:
              precision    recall  f1-score   support

       False       0.54      0.15      0.24     51857
        True       0.81      0.96      0.88    193067

    accuracy                           0.79    244924
   macro avg       0.67      0.56      0.56    244924
weighted avg       0.75      0.79      0.74    244924

{
    "accuracy": 0.7925478924074407,
    "precision": 0.8091427876027346,
    "recall": 0.9642766500748445,
    "f1": 0.8799243766985703
}
Comparison between non-control model and full model (negatives indicate non-control is better):
{
    "recall": -0.017400031407072802,
    "accuracy": 0.07935114566151125,
    "f1": 0.04104201229037907,
    "precision": 0.08729361241099609
}


### Non-control model without intra-project factors

In [12]:
# Creates measured information model.
non_control_fields_wo_intra = [field for field in non_control_fields
                               if not re.match(r'.*Intra.*', field)]
non_control_wo_intra_rf_model, non_control_wo_intra_rf_metrics = create_model(
    train[non_control_fields_wo_intra],
    train_dependent,
    test[non_control_fields_wo_intra],
    test_dependent
)

print(json.dumps(non_control_rf_metrics, indent=4))
print("Comparison between non-control model (without intra-project factors) and full model (negatives indicate non-control is better):")
diff = subtract_dict(full_rf_metrics, non_control_rf_metrics)
print(json.dumps(diff, indent=4))


random_state=44
metrics={'accuracy': 0.7818057846515654, 'precision': 0.7989056438975518, 'recall': 0.9664727788798707, 'f1': 0.874736597840278}

Confusion matrix:
[[  4889  46968]
 [  6473 186594]]

Classification report:
              precision    recall  f1-score   support

       False       0.43      0.09      0.15     51857
        True       0.80      0.97      0.87    193067

    accuracy                           0.78    244924
   macro avg       0.61      0.53      0.51    244924
weighted avg       0.72      0.78      0.72    244924

{
    "accuracy": 0.7925478924074407,
    "precision": 0.8091427876027346,
    "recall": 0.9642766500748445,
    "f1": 0.8799243766985703
}
Comparison between non-control model (without intra-project factors) and full model (negatives indicate non-control is better):
{
    "recall": -0.017400031407072802,
    "accuracy": 0.07935114566151125,
    "f1": 0.04104201229037907,
    "precision": 0.08729361241099609
}


## Ablation Study

In [17]:
from typing import Iterator


def ablation_study(
    train_predictors: pd.DataFrame,
    train_labels: pd.Series,
    test_predictors: pd.DataFrame,
    test_labels: pd.Series,
    evaluated_fields: Iterator[str],
):
    f1_differences_per_feature = {}

    for excluded_feature in evaluated_fields:
        print(f"Creating model without: {excluded_feature}.")
        _train_predictors = train_predictors.drop(excluded_feature, axis=1)
        _test_predictors = test_predictors.drop(excluded_feature, axis=1)

        _, excl_rf_metrics = create_model(
            _train_predictors, train_labels, _test_predictors, test_labels
        )

        diff = subtract_dict(full_rf_metrics, excl_rf_metrics)

        f1_differences_per_feature[excluded_feature] = {
            'Weighted metrics': excl_rf_metrics,
            'Weighted metrics difference': diff,
        }

    return f1_differences_per_feature


f1_diffs_per_feature = ablation_study(
    train_predictors, train_dependent,
    test_predictors, test_dependent,
    independent_fields
)

print("Performance metrics ablation study vs. the full model (negatives suggest the ablation model is better):")
print(f1_diffs_per_feature)


Creating model without: ControlIntegratedBySameUser.


ValueError: Cannot specify both 'labels' and 'index'/'columns'

## Partial Dependence Plots

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
from matplotlib import pyplot as plt


def create_partial_dependence_plots(
        rf_model: RandomForestRegressor,
        used_predictors: pd.DataFrame,
        used_labels: pd.Series,
        model_name: str = ""):

    # Collective partial dependence plot.
    PartialDependenceDisplay.from_estimator(
        rf_model, used_predictors, used_labels)

    fig = plt.gcf()
    axs = fig.axes

    lines = []
    for ax in axs:
        __lines = list([(line.get_xdata(), line.get_ydata())
                        for line in ax.lines])
        lines.extend(__lines)

    for (x, y), label in zip(lines, used_predictors.columns):
        plt.clf()
        plt.plot(x, y, linestyle='-', color='#e69d00')
        plt.xlabel(label)
        plt.ylabel('Partial Dependence')
        plt.tight_layout()
        output_path = f"{figure_base_path}/partial-dependence/{model_name}/{label}.png"
        safe_save_fig(output_path)


In [ ]:
create_partial_dependence_plots(
    full_rf_model,
    test_predictors,
    test_dependent,
    model_name="full_model"
)
